In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import pickle  
import os
from pycaret.classification import load_model, predict_model
import datetime
import numpy as np
from scipy.stats import entropy

setup_function():
- load out-of-sample data
- load all models

In [ ]:
#Loading different datasets
#df = pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='quiruelas', engine='openpyxl')
#df = pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='v_higueras', engine='openpyxl')
#df = pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='Alberite', engine='openpyxl')
#df = pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='Paternanbidea', engine='openpyxl')
#df = pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='Can_Gambus', engine='openpyxl')
#df = pd.read_excel('/home/dsg/VORTEX_FINAL/PRODUCTION/DATA/real_world/real_world_data.xlsx',sheet_name='CatalonianSites', engine='openpyxl')
df.info()


In [ ]:
#Loading models
full_model = load_model(model_path)
destilled_model = load_model(model_path)
#specific
VdH = load_model(model_path)
PQ = load_model(model_path)
French = load_model(model_path)

### Prediction_function():
- 

In [ ]:

def prediction(model_path, excel_path, exclude_columns=None, output_column='predictions'):
    """
    Carga un archivo Excel,
    hace predicciones usando un modelo pre-entrenado,
    y guarda los resultados en un nuevo archivo Excel o CSV.

    :param model_path: Ruta al archivo del modelo 
    :param excel_path: Ruta al archivo con los datos de entrada
    :param exclude_columns: Lista de columnas a excluir como entrada (opcional)
    :param output_column: Nombre de la columna para las predicciones (por defecto: 'predictions')
    :return: Ruta al archivo de salida con las predicciones
    """
    # Loading  the model usando PyCaret
    modelo = load_model(model_path)

    # Cargar los datos
    df = pd.read_excel(excel_path)


    # Excluir columnas si se especifican
    if exclude_columns:
        df = df.drop(columns=[col for col in exclude_columns], errors='ignore')

    # Hacer predicciones usando PyCaret
    predictions = predict_model(model, data=df,raw_score=True)
    
    # El resultado de predict_model ya incluye las predicciones, 
    # generalmente en una columna llamada 'Label' o similar
    # Renombrar la columna de predicciones si es necesario
    if 'Label' in predicciones.columns:
        predicciones = predicciones.rename(columns={'Label': output_column})
    elif 'prediction_label' in predicciones.columns:
        predicciones = predicciones.rename(columns={'prediction_label': output_column})
    else:
        # Si la columna de predicciones tiene otro nombre, ajusta esto según sea necesario
        pass

   
    # Create name for the output file
    base_name = os.path.splitext(excel_path)[0]
    current_date = datetime.datetime.now().strftime("%Y%m%d")
    output_path = f"{base_name}_{current_date}_predictions.xlsx"
   

    # save results
    predictions.to_excel(output_path, index=False)
    predictions.to_csv(output_path, index=False)

    return output_path

In [ ]:

# Usage:
model_path = '20240827_SinCaDestilledVortex2SMOTECalibrated'
excel_path = "./DATA/VARISCITA ROMANA100.xlsx"
exclude_columns = [] 

try:
    output_file = predict_from_excel(model_path, excel_path, exclude_columns)
    print(f"Predictions saved in: {output_file}")
except Exception as e:
    print(f"Error processing the file: {str(e)}")


class ProvenanceAndUncertaintyAnalysis:

- def predicitons_uncertainty_analysis():
  - Average confidence per site
  - Median entropy per site
  - Median probabilities per site

- Provenance determination
  - Confidence treshold >0.7
  - Majority vote sourcing
  - Homogeneity
 


 

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import entropy

def predictions_uncertainty_analysis(prediction_df, confidence_threshold=0.7):


    """
    Performs uncertainty analysis per site

    Args:
        prediction_df: df with probabilistic predictions
        confidence_treshold: threshold to mark the prediction as uncertain
     
    """
    # Convert score columns to numbers
    score_cols = ['prediction_score_CT', 'prediction_score_PCM', 'prediction_score_PDLC']
    for col in score_cols:
        df[col] = df[col].str.replace(',', '.').astype(float)
    
    # Obtener probabilidades
    probas = df[score_cols].values
    
    # Obtener predicción y confianza
    predictions = df['predicciones'].values
    confidences = np.max(probas, axis=1)
    
    # Marcar predicciones por debajo del umbral como inciertas
    uncertain_mask = confidences < confidence_threshold
    predictions_with_uncertainty = predictions.copy()
    predictions_with_uncertainty[uncertain_mask] = 'uncertain'
    
    # Calcular entropía 
    entropies = np.array([entropy(probs, base=2) for probs in probas])
    
    # Crear DataFrame con resultados
    results_df = pd.DataFrame({
        'id': df['id'],
        'Site': df['Site'],
        'Original_predictions': predictions,
        'Uncertainty_treshold_predictions': predictions_with_uncertainty,
        'entropy': entropies
    })
    
    # Imprimir métricas básicas
    n_uncertain = np.sum(uncertain_mask)
    print(f"Uncertain predictions: {n_uncertain}/{len(df)} ({(n_uncertain/len(df)*100):.1f}%)")
    print(f"Median entropies: {entropies.median():.3f}")
    
    
    # Calcular y mostrar la mediana de entropia por sitio
    print("\nMedian entropy per site:")
    entropy_median_by_site = results_df.groupby('Site')['entropy'].median()
    for site, median in entropy_median_by_site.items():
        print(f"{site}: {median:.3f}")
    
    return results_df

Plotting function:
- Create uncertain plot
- Save it in plots
- close the plot

In [ ]:
def plot_function():